# Importing the necessary libraries

In [1]:
import yaml
import os
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType
import json

# Transforming the drivers table:

## getting all possible keys in all yaml files

In [2]:
folder_path = "/home/floppabox/f1/f1db/src/data/drivers"
max_length = 0
keys = []
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        data=yaml.safe_load(file)
        if max_length<=len(list(data.keys())):
            max_length = len(list(data.keys()))
            keys = list(data.keys())

print(f'there are {max_length} keys in drivers:')
print(keys)


there are 15 keys in drivers:
['id', 'name', 'firstName', 'lastName', 'fullName', 'abbreviation', 'permanentNumber', 'gender', 'dateOfBirth', 'dateOfDeath', 'placeOfBirth', 'countryOfBirthCountryId', 'nationalityCountryId', 'secondNationalityCountryId', 'familyRelationships']


In [3]:
spark = SparkSession.builder.appName("YAML to CSV").getOrCreate()

24/12/04 08:38:26 WARN Utils: Your hostname, Obuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/12/04 08:38:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/04 08:38:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
data = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        content_drivers=yaml.safe_load(file)
        record ={}
        for key in keys:
            if key != 'familyRelationships':
                record[key]= content_drivers.get(key)
        data.append(record)

print(data)


In [4]:
data = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        content_drivers=yaml.safe_load(file)
        record ={}

        if 'familyRelationships' in content_drivers.keys():
            for rel in content_drivers['familyRelationships']:
                record['driverId']=content_drivers.get('id')
                record['relationId']=rel.get('driverId')
                record['type']=rel.get('type')
            data.append(record)

print(data)

[{'driverId': 'pedro-rodriguez', 'relationId': 'ricardo-rodriguez', 'type': 'SIBLING'}, {'driverId': 'max-papis', 'relationId': 'emerson-fittipaldi', 'type': 'PARENT_IN_LAW'}, {'driverId': 'jimmy-stewart', 'relationId': 'jackie-stewart', 'type': 'SIBLING'}, {'driverId': 'jacques-villeneuve', 'relationId': 'jacques-villeneuve-sr', 'type': 'PARENTS_SIBLING'}, {'driverId': 'max-verstappen', 'relationId': 'jos-verstappen', 'type': 'PARENT'}, {'driverId': 'mario-andretti', 'relationId': 'michael-andretti', 'type': 'CHILD'}, {'driverId': 'eddie-russo', 'relationId': 'eddie-russo', 'type': 'PARENTS_SIBLING'}, {'driverId': 'jules-bianchi', 'relationId': 'lucien-bianchi', 'type': 'GRANDPARENTS_SIBLING'}, {'driverId': 'nico-rosberg', 'relationId': 'keke-rosberg', 'type': 'PARENT'}, {'driverId': 'ian-scheckter', 'relationId': 'jody-scheckter', 'type': 'SIBLING'}, {'driverId': 'dick-rathmann', 'relationId': 'jim-rathmann', 'type': 'SIBLING'}, {'driverId': 'christian-fittipaldi', 'relationId': 'pie

In [5]:
dataframe = spark.createDataFrame(data)

dataframe.printSchema()
dataframe.count()

root
 |-- driverId: string (nullable = true)
 |-- relationId: string (nullable = true)
 |-- type: string (nullable = true)



64